In [ ]:
import pandas as pd
import numpy as np
import jieba
import pyecharts
from collections import Counter
from pyecharts.charts import Line
from pyecharts.charts import Bar
from datetime import datetime

In [ ]:
df = pd.read_json('cnblog_article.json', encoding='utf-8')
# df = df[['postdate', 'views']]
df = df.drop(['title', 'sumary', 'posttime', 'views'], axis=1)
df['count'] = 1
df

In [ ]:
df['postdate'] = pd.to_datetime(df['postdate'])
df = df.set_index('postdate')

In [ ]:
result = df.resample('M').sum()
result

In [ ]:
x1, y1 = [], []
for i in result.index:
    x1.append(i)
for i in result.values:
    y1.append(i[0])
print(x1, y1)

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Bar

c = (
    Bar()
    .add_xaxis(x1)
    .add_yaxis("商家A", y1)
    .set_global_opts(title_opts=opts.TitleOpts(title="Bar-基本示例", subtitle="我是副标题"))
    .render_notebook()
)
c

------------

In [ ]:
week_data = []
for i in df_json['postdate']:
    week_data.append(datetime.strptime(i, "%Y-%m-%d").weekday())

In [ ]:
time_data = []
for i in df_json['posttime']:
    time_data.append(i.split(":")[0])

In [ ]:
tc = Counter(time_data)
# time_count_date = [tc[i] for i in tc]
# time_count_date
tc

In [ ]:
from datetime import datetime
map_data = []
for item in df_json.values:
    week = datetime.strptime(item[2], "%Y-%m-%d").weekday()
    print(week)

In [ ]:
line = Bar()
# x_data = [i for i in date_counter]
# y_data = [date_counter[i] for i in date_counter]
line.add_xaxis(time_data)
line.add_yaxis("count", time_count_date)
line.render_notebook()

In [214]:
import pandas as pd
import numpy as np
import jieba
import pyecharts
from collections import Counter
from pyecharts.charts import WordCloud
from pyecharts.charts import Line

# 获取所有字段存为一个字符串
def get_text(json_file, column='title'):
    df_json = pd.read_json(json_file, encoding='utf-8')
    text = ''
    for i in df_json[column]:
        text += i
    return text

# 去停用词，使用jieba分词
def split_word(text):
    word_list = list(jieba.cut(text))
    # 去掉一些无意义的词和符号，我这里自己整理了停用词库
    with open('stop_word.txt', encoding='utf-8') as f:
        meaningless_word = f.read().splitlines()
        # print(meaningless_word)
    result = []
    # 筛选词语
    for i in word_list:
        if i not in meaningless_word:
            result.append(i.replace(' ', ''))
    return result

# 词频统计
def word_counter(words):
    # 使用Count计数方法
    words_counter = Counter(words)
    # 将Counter类型转换为列表
    words_list = words_counter.most_common(100)
    return words_list

# 生成词云
def create_wordcloud(json_file, title='词云', column='title'):
    text = get_text(json_file, column=column)
    clear_word = split_word(text)
    data = word_counter(clear_word)
    wd = WordCloud()
    wd.add(series_name=title, data_pair=data, word_size_range=[40, 150])
    wd.set_global_opts(title_opts=opts.TitleOpts(title="你的文章词云", subtitle="基于你的博客数据生成", title_textstyle_opts=opts.TextStyleOpts(font_size=23)), tooltip_opts=opts.TooltipOpts(is_show=True))
    # wd.render_notebook()
    wd.render('topic_wordcloud.html')

# 生成折线图
def create_postdate_line(json_file, title='折线图', column='postdate'):
    df_json = pd.read_json('cnblog_article.json', encoding='utf-8')
    postdate_month_list = []
    for i in df_json[column]:
        postdate_month_list.append('-'.join(i.split('-')[:-1]))
    date_counter = Counter(postdate_month_list)
    line = Line()
    x_data = [i for i in date_counter]
    y_data = [date_counter[i] for i in date_counter]
    line.add_xaxis(x_data)
    line.add_yaxis(series_name="发文数量", y_axis=y_data)
    line.set_global_opts(title_opts=opts.TitleOpts(title="你的发文数量", subtitle="基于你的博客数据生成"))
    line.render('postdate_line.html')

In [215]:
create_wordcloud('cnblog_article.json', title='你的创作领域词云', column='title')
create_postdate_line('cnblog_article.json', title='发文时间线', column='postdate')